In [1]:
data Expr a
    = Lit a
    | Var Int
    | Abs (Expr a)
    | App (Expr a) (Expr a)
    deriving (Show)

In [2]:
{-# LANGUAGE TypeFamilies, DataKinds, ConstraintKinds #-}
{-# LANGUAGE GADTs, EmptyCase, StandaloneDeriving #-}
{-# LANGUAGE TypeOperators, PatternSynonyms #-}
{-# LANGUAGE FlexibleInstances, FlexibleContexts #-}

In [3]:
data ExpX i a
    = LitX (XLit i a) a
    | VarX (XVar i a)
    | AbsX (XAbs i a) (ExpX i a)
    | AppX (XApp i a) (ExpX i a) (ExpX i a)
    | ExpX (XExp i a)

type family XLit i a
type family XVar i a
type family XAbs i a
type family XApp i a
type family XExp i a

In [4]:
import Data.Void

void :: Void
void = error "Attempt to evaluate void"

type ExpUD a = ExpX UD a
data UD
type instance XLit UD a = Void
type instance XVar UD a = Int
type instance XAbs UD a = Void
type instance XApp UD a = Void
type instance XExp UD a = Void

pattern LitUD :: a -> ExpUD a
pattern LitUD a <- LitX _ a
    where LitUD a = LitX void a
pattern VarUD :: Int -> ExpUD a
pattern VarUD i <- VarX i
    where VarUD i = VarX i
pattern AbsUD :: ExpUD a -> ExpUD a
pattern AbsUD a <- AbsX _ a
    where AbsUD a = AbsX void a
pattern AppUD :: ExpUD a -> ExpUD a -> ExpUD a
pattern AppUD f a <- AppX _ f a
    where AppUD f a = AppX void f a

In [5]:
type ExpAnn a = ExpX Ann a
data Ann
type instance XLit Ann a = Void
type instance XVar Ann a = (String, Int)
type instance XAbs Ann a = String
type instance XApp Ann a = Void
type instance XExp Ann a = Void

pattern LitAnn :: a -> ExpAnn a
pattern LitAnn a <- LitX _ a
    where LitAnn a = LitX void a
pattern VarAnn :: String -> Int -> ExpAnn a
pattern VarAnn s i <- VarX (s,i)
    where VarAnn s i = VarX (s, i)
pattern AbsAnn :: String -> ExpAnn a -> ExpAnn a
pattern AbsAnn s a <- AbsX s a
    where AbsAnn s a = AbsX s a
pattern AppAnn :: ExpAnn a -> ExpAnn a -> ExpAnn a
pattern AppAnn f a <- AppX _ f a
    where AppAnn f a = AppX void f a

In [6]:
type ExpLet a = ExpX Let a
data Let
type instance XLit Let a = Void
type instance XVar Let a = String
type instance XAbs Let a = String
type instance XApp Let a = Void
type instance XExp Let a = (String, ExpLet a, ExpLet a)

pattern LitLet :: a -> ExpLet a
pattern LitLet a <- LitX _ a
    where LitLet a = LitX void a
pattern VarLet :: String -> ExpLet a
pattern VarLet s <- VarX s
    where VarLet s = VarX s
pattern AbsLet :: String -> ExpLet a -> ExpLet a
pattern AbsLet s a <- AbsX s a
    where AbsLet s a = AbsX s a
pattern AppLet :: ExpLet a -> ExpLet a -> ExpLet a
pattern AppLet f a <- AppX _ f a
    where AppLet f a = AppX void f a
pattern LetLet n v e <- ExpX (n,v,e)

In [7]:
import qualified Data.Map.Strict as Map

type Env = Map.Map String Int

desugar :: Env -> ExpLet a -> ExpAnn a
desugar env expr = case expr of
    LitLet a -> LitAnn a
    VarLet name -> VarAnn name (env Map.! name)
    AbsLet name expr' -> let
        env'  = Map.map succ env
        env'' = Map.insert name 0 env'
        in AbsAnn name (desugar env'' expr')
    AppLet f x -> AppAnn (desugar env f) (desugar env x)
    LetLet n v expr' -> desugar env (AppLet (AbsLet n expr') v)

anonymise :: ExpAnn a -> ExpUD a
anonymise expr = case expr of
    LitAnn a -> LitUD a
    VarAnn _ i -> VarUD i
    AbsAnn _ e -> AbsUD (anonymise e)
    AppAnn f x -> AppUD (anonymise f) (anonymise x)

eval :: [a] -> ExpUD a -> a
eval env expr = case expr of
    LitUD a -> a
    VarUD i -> env !! i
    AbsUD f -> eval env f
    AppUD f x -> let
        x' = eval env x
        in eval (x':env) f

In [8]:
deriving instance (Show a) => (Show (ExpLet a))
deriving instance (Show a) => (Show (ExpAnn a))
deriving instance (Show a) => (Show (ExpUD a))

identity = AbsLet "i" (VarLet "i")
konst = (AbsLet "x" (AbsLet "y" (VarLet "x")))


eval [] . anonymise . desugar Map.empty $ AppLet (AppLet konst (LitLet 1)) (LitLet 2)

2